This notebook splits the full review video (review_video.mp4) into five horizontal sections, and then splits each section into 20 second clips. The resulting video clips were manually reviewed by the authors to correct the buffalo count initially generated by the automated detection and counting processes.

In [9]:
import cv2
import math

In [10]:
# Read the input video
video_path = "figures_videos/review_video.mp4"
cap = cv2.VideoCapture(video_path)

# Get the frame size and duration
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
duration = math.ceil(frame_count / fps)

# Divide the frame into 5 non-overlapping side-by-side areas
area_width = int(frame_width / 5)
area_height = frame_height
areas = [(i * area_width, 0, area_width, area_height) for i in range(5)]

# Initialize the video writers for each area
writers = []
for i in range(5):
    writer = cv2.VideoWriter(f"output_video_{i}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (area_width, area_height))
    writers.append(writer)

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Save each frame to its corresponding area
    for i, area in enumerate(areas):
        x, y, w, h = area
        area_frame = frame[y:y+h, x:x+w]
        writers[i].write(area_frame)

    # Break the video into 20 second clips
    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    if current_frame % (fps * 20) == 0:
        for writer in writers:
            writer.release()
        writers = []
        for i in range(5):
            writer = cv2.VideoWriter(f"figures_videos/manual_review_clips/output_video_{i}_{current_frame // (fps * 20)}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (area_width, area_height))
            writers.append(writer)

# Release the video capture and writers
cap.release()
for writer in writers:
    writer.release()